In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from pyscf import gto, scf, fci

# Function to define molecular geometry for O2
def get_o2_coords(d):
    """
    Returns coordinates for O₂ diatomic molecule with O at origin
    """
    return [["O", [0.0, 0.0, 0.0]], 
            ["O", [0.0, 0.0, d]]]

# Function to generate the PySCF molecule
def get_pyscf_mol(coords, charge, spin, basis):
    """
    Create a PySCF molecule object with the given geometry, charge, spin, and basis set.
    """
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")
    mol = gto.Mole()
    mol.atom = atom_spec
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol

# Function to generate the Qiskit problem
def get_qiskit_problem(coords, charge, spin, basis):
    """
    Generate the Qiskit Nature problem for the given geometry, charge, spin, and basis set.
    """
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")
    driver = PySCFDriver(atom=atom_spec, unit=DistanceUnit.ANGSTROM, charge=charge, spin=spin, basis=basis)
    return driver.run()

# Function to run PySCF UHF calculation
def run_uhf_pyscf(mol):
    """
    Perform Unrestricted Hartree-Fock (UHF) calculation using PySCF.
    """
    mf = scf.UHF(mol)
    ehf = mf.kernel()
    return ehf

# Function to run PySCF FCI calculation
def run_fci_pyscf(mol):
    """
    Perform Full Configuration Interaction (FCI) calculation using PySCF.
    """
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    efci, _ = cisolver.kernel()
    return efci

# Function to run VQE using Qiskit
def run_vqe_cpu(problem, mapper):
    """
    Perform VQE calculation using Qiskit with the UCCSD ansatz.
    """
    # Use standard Qiskit Estimator (CPU-based)
    estimator = Estimator()
    
    optimizer = COBYLA(maxiter=200)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)
    
    # Better initial point (small random values)
    np.random.seed(42)
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy

# --- Main Calculation Setup ---
basis = 'sto-3g'
charge = 0  # Neutral system
spin = 2    # Triplet state (open-shell O₂)
dists = np.linspace(0.4, 4.0, 50)  # O-O distances in Å

vqe_energies, uhf_energies, fci_energies = [], [], []

# Loop through different O-O distances
for i, d in enumerate(dists):
    print(f"Calculating O₂ at d = {d:.2f} Å ({i+1}/{len(dists)})")
    coords = get_o2_coords(d)
    
    # Classical reference calculations
    mol = get_pyscf_mol(coords, charge, spin, basis)
    uhf_energy = run_uhf_pyscf(mol)
    fci_energy = run_fci_pyscf(mol)
    
    # VQE calculation
    problem = get_qiskit_problem(coords, charge, spin, basis)
    mapper = ParityMapper()
    vqe_energy = run_vqe_cpu(problem, mapper)
    
    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)
    print(f"  VQE={vqe_energy:.6f} | UHF={uhf_energy:.6f} | FCI={fci_energy:.6f}")

# Plot potential energy curves
plt.figure(figsize=(9, 5))

# Plot curves
plt.plot(dists, vqe_energies, '-o', label='VQE', color='green', linewidth=1, markersize=4)
plt.plot(dists, uhf_energies, '-s', label='UHF', color='red', linewidth=1, markersize=4)
plt.plot(dists, fci_energies, '--', label='FCI', color='black', linewidth=1)

plt.xlabel("O-O Distance (Å)", fontsize=12)
plt.ylabel("Energy (Hartree)", fontsize=12)
plt.title("O₂ Potential Energy Surface", fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Find minimum energy and corresponding distance
min_idx = np.argmin(vqe_energies)
min_energy = vqe_energies[min_idx]
eq_distance = dists[min_idx]

print(f"O₂ ground state energy (VQE): {min_energy:.6f} Hartree")
print(f"Equilibrium O-O distance: {eq_distance:.3f} Å")
print(f"Energy difference VQE-FCI: {(min_energy-fci_energies[min_idx])*1000:.2f} mHartree")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from pyscf import gto, scf, fci

# Function to define molecular geometry for O2
def get_o2_coords(d):
    """
    Returns coordinates for O₂ diatomic molecule with O at origin
    """
    return [["O", [0.0, 0.0, 0.0]], 
            ["O", [0.0, 0.0, d]]]

# Function to generate the PySCF molecule
def get_pyscf_mol(coords, charge, spin, basis):
    """
    Create a PySCF molecule object with the given geometry, charge, spin, and basis set.
    """
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")
    mol = gto.Mole()
    mol.atom = atom_spec
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol

# Function to generate the Qiskit problem
def get_qiskit_problem(coords, charge, spin, basis):
    """
    Generate the Qiskit Nature problem for the given geometry, charge, spin, and basis set.
    """
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")
    driver = PySCFDriver(atom=atom_spec, unit=DistanceUnit.ANGSTROM, charge=charge, spin=spin, basis=basis)
    return driver.run()

# Function to run PySCF UHF calculation
def run_uhf_pyscf(mol):
    """
    Perform Unrestricted Hartree-Fock (UHF) calculation using PySCF.
    """
    mf = scf.UHF(mol)
    ehf = mf.kernel()
    return ehf

# Function to run PySCF FCI calculation
def run_fci_pyscf(mol):
    """
    Perform Full Configuration Interaction (FCI) calculation using PySCF.
    """
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    efci, _ = cisolver.kernel()
    return efci

# Function to run VQE using Qiskit
def run_vqe_cpu(problem, mapper):
    """
    Perform VQE calculation using Qiskit with the UCCSD ansatz.
    """
    # Use standard Qiskit Estimator (CPU-based)
    estimator = Estimator()
    
    optimizer = COBYLA(maxiter=200)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)
    
    # Better initial point (small random values)
    np.random.seed(42)
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy

# --- Main Calculation Setup ---
basis = 'aug-cc-pVTZ'
charge = 0  # Neutral system
spin = 2    # Triplet state (open-shell O₂)
dists = np.linspace(0.4, 4.0, 50)  # O-O distances in Å

vqe_energies, uhf_energies, fci_energies = [], [], []

# Loop through different O-O distances
for i, d in enumerate(dists):
    print(f"Calculating O₂ at d = {d:.2f} Å ({i+1}/{len(dists)})")
    coords = get_o2_coords(d)
    
    # Classical reference calculations
    mol = get_pyscf_mol(coords, charge, spin, basis)
    uhf_energy = run_uhf_pyscf(mol)
    fci_energy = run_fci_pyscf(mol)
    
    # VQE calculation
    problem = get_qiskit_problem(coords, charge, spin, basis)
    mapper = ParityMapper()
    vqe_energy = run_vqe_cpu(problem, mapper)
    
    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)
    print(f"  VQE={vqe_energy:.6f} | UHF={uhf_energy:.6f} | FCI={fci_energy:.6f}")

# Plot potential energy curves
plt.figure(figsize=(9, 5))

# Plot curves
plt.plot(dists, vqe_energies, '-o', label='VQE', color='green', linewidth=1, markersize=4)
plt.plot(dists, uhf_energies, '-s', label='UHF', color='red', linewidth=1, markersize=4)
plt.plot(dists, fci_energies, '--', label='FCI', color='black', linewidth=1)

plt.xlabel("O-O Distance (Å)", fontsize=12)
plt.ylabel("Energy (Hartree)", fontsize=12)
plt.title("O₂ Potential Energy Surface (aug-cc-pVTZ)", fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Find minimum energy and corresponding distance
min_idx = np.argmin(vqe_energies)
min_energy = vqe_energies[min_idx]
eq_distance = dists[min_idx]

print(f"O₂ ground state energy (VQE): {min_energy:.6f} Hartree")
print(f"Equilibrium O-O distance: {eq_distance:.3f} Å")
print(f"Energy difference VQE-FCI: {(min_energy-fci_energies[min_idx])*1000:.2f} mHartree")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from pyscf import gto, scf, fci

# Function to define molecular geometry for CH4
def get_ch4_coords(bond_length):
    """
    Returns coordinates for CH₄ (methane) molecule with tetrahedral geometry.
    The bond length is varied for all C-H bonds.
    """
    coords = [
        ["C", [0.0, 0.0, 0.0]],  # Carbon at origin
        ["H", [bond_length, 0.0, 0.0]],  # Hydrogen 1
        ["H", [-bond_length / 2, bond_length * (3 ** 0.5) / 2, 0.0]],  # Hydrogen 2
        ["H", [-bond_length / 2, -bond_length * (3 ** 0.5) / 2, 0.0]],  # Hydrogen 3
        ["H", [0.0, 0.0, bond_length]],  # Hydrogen 4
    ]
    return coords

# Function to generate the PySCF molecule
def get_pyscf_mol(coords, charge, spin, basis):
    """
    Create a PySCF molecule object with the given geometry, charge, spin, and basis set.
    """
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")
    mol = gto.Mole()
    mol.atom = atom_spec
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol

# Function to generate the Qiskit problem
def get_qiskit_problem(coords, charge, spin, basis):
    """
    Generate the Qiskit Nature problem for the given geometry, charge, spin, and basis set.
    """
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")
    driver = PySCFDriver(atom=atom_spec, unit=DistanceUnit.ANGSTROM, charge=charge, spin=spin, basis=basis)
    return driver.run()

# Function to run PySCF UHF calculation
def run_uhf_pyscf(mol):
    """
    Perform Unrestricted Hartree-Fock (UHF) calculation using PySCF.
    """
    mf = scf.UHF(mol)
    ehf = mf.kernel()
    return ehf

# Function to run PySCF FCI calculation
def run_fci_pyscf(mol):
    """
    Perform Full Configuration Interaction (FCI) calculation using PySCF.
    """
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    efci, _ = cisolver.kernel()
    return efci

# Function to run VQE using Qiskit
def run_vqe_cpu(problem, mapper):
    """
    Perform VQE calculation using Qiskit with the UCCSD ansatz.
    """
    # Use standard Qiskit Estimator (CPU-based)
    estimator = Estimator()
    
    optimizer = COBYLA(maxiter=200)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)
    
    # Better initial point (small random values)
    np.random.seed(42)
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy

# --- Main Calculation Setup ---
basis = 'sto-3g'
charge = 0  # Neutral molecule
spin = 0    # Singlet state (closed-shell CH₄)
dists = np.linspace(0.4, 4.0, 50)  # C-H bond distances in Å

vqe_energies, uhf_energies, fci_energies = [], [], []

# Loop through different C-H bond lengths
for i, d in enumerate(dists):
    print(f"Calculating CH₄ at C-H bond length = {d:.2f} Å ({i+1}/{len(dists)})")
    coords = get_ch4_coords(d)
    
    # Classical reference calculations
    mol = get_pyscf_mol(coords, charge, spin, basis)
    uhf_energy = run_uhf_pyscf(mol)
    fci_energy = run_fci_pyscf(mol)
    
    # VQE calculation
    problem = get_qiskit_problem(coords, charge, spin, basis)
    mapper = ParityMapper()
    vqe_energy = run_vqe_cpu(problem, mapper)
    
    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)
    print(f"  VQE={vqe_energy:.6f} | UHF={uhf_energy:.6f} | FCI={fci_energy:.6f}")

# Plot potential energy curves
plt.figure(figsize=(9, 5))

# Plot curves
plt.plot(dists, vqe_energies, '-o', label='VQE', color='green', linewidth=1, markersize=4)
plt.plot(dists, uhf_energies, '-s', label='UHF', color='red', linewidth=1, markersize=4)
plt.plot(dists, fci_energies, '--', label='FCI', color='black', linewidth=1)

plt.xlabel("C-H Bond Length (Å)", fontsize=12)
plt.ylabel("Energy (Hartree)", fontsize=12)
plt.title("CH₄ Potential Energy Surface (STO-3G)", fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Find minimum energy and corresponding bond length
min_idx = np.argmin(vqe_energies)
min_energy = vqe_energies[min_idx]
eq_distance = dists[min_idx]

print(f"CH₄ ground state energy (VQE): {min_energy:.6f} Hartree")
print(f"Equilibrium C-H bond length: {eq_distance:.3f} Å")
print(f"Energy difference VQE-FCI: {(min_energy-fci_energies[min_idx])*1000:.2f} mHartree")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from pyscf import gto, scf, fci

# Function to define molecular geometry for CH4
def get_ch4_coords(bond_length):
    """
    Returns coordinates for CH₄ (methane) molecule with tetrahedral geometry.
    The bond length is varied for all C-H bonds.
    """
    coords = [
        ["C", [0.0, 0.0, 0.0]],  # Carbon at origin
        ["H", [bond_length, 0.0, 0.0]],  # Hydrogen 1
        ["H", [-bond_length / 2, bond_length * (3 ** 0.5) / 2, 0.0]],  # Hydrogen 2
        ["H", [-bond_length / 2, -bond_length * (3 ** 0.5) / 2, 0.0]],  # Hydrogen 3
        ["H", [0.0, 0.0, bond_length]],  # Hydrogen 4
    ]
    return coords

# Function to generate the PySCF molecule
def get_pyscf_mol(coords, charge, spin, basis):
    """
    Create a PySCF molecule object with the given geometry, charge, spin, and basis set.
    """
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")
    mol = gto.Mole()
    mol.atom = atom_spec
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol

# Function to generate the Qiskit problem
def get_qiskit_problem(coords, charge, spin, basis):
    """
    Generate the Qiskit Nature problem for the given geometry, charge, spin, and basis set.
    """
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")
    driver = PySCFDriver(atom=atom_spec, unit=DistanceUnit.ANGSTROM, charge=charge, spin=spin, basis=basis)
    return driver.run()

# Function to run PySCF UHF calculation
def run_uhf_pyscf(mol):
    """
    Perform Unrestricted Hartree-Fock (UHF) calculation using PySCF.
    """
    mf = scf.UHF(mol)
    ehf = mf.kernel()
    return ehf

# Function to run PySCF FCI calculation
def run_fci_pyscf(mol):
    """
    Perform Full Configuration Interaction (FCI) calculation using PySCF.
    """
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    efci, _ = cisolver.kernel()
    return efci

# Function to run VQE using Qiskit
def run_vqe_cpu(problem, mapper):
    """
    Perform VQE calculation using Qiskit with the UCCSD ansatz.
    """
    # Use standard Qiskit Estimator (CPU-based)
    estimator = Estimator()
    
    optimizer = COBYLA(maxiter=200)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)
    
    # Better initial point (small random values)
    np.random.seed(42)
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy

# --- Main Calculation Setup ---
basis = 'cc-pVTZ'
charge = 0  # Neutral molecule
spin = 0    # Singlet state (closed-shell CH₄)
dists = np.linspace(0.4, 4.0, 50)  # C-H bond distances in Å

vqe_energies, uhf_energies, fci_energies = [], [], []

# Loop through different C-H bond lengths
for i, d in enumerate(dists):
    print(f"Calculating CH₄ at C-H bond length = {d:.2f} Å ({i+1}/{len(dists)})")
    coords = get_ch4_coords(d)
    
    # Classical reference calculations
    mol = get_pyscf_mol(coords, charge, spin, basis)
    uhf_energy = run_uhf_pyscf(mol)
    fci_energy = run_fci_pyscf(mol)
    
    # VQE calculation
    problem = get_qiskit_problem(coords, charge, spin, basis)
    mapper = ParityMapper()
    vqe_energy = run_vqe_cpu(problem, mapper)
    
    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)
    print(f"  VQE={vqe_energy:.6f} | UHF={uhf_energy:.6f} | FCI={fci_energy:.6f}")

# Plot potential energy curves
plt.figure(figsize=(9, 5))

# Plot curves
plt.plot(dists, vqe_energies, '-o', label='VQE', color='green', linewidth=1, markersize=4)
plt.plot(dists, uhf_energies, '-s', label='UHF', color='red', linewidth=1, markersize=4)
plt.plot(dists, fci_energies, '--', label='FCI', color='black', linewidth=1)

plt.xlabel("C-H Bond Length (Å)", fontsize=12)
plt.ylabel("Energy (Hartree)", fontsize=12)
plt.title("CH₄ Potential Energy Surface (STO-3G)", fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Find minimum energy and corresponding bond length
min_idx = np.argmin(vqe_energies)
min_energy = vqe_energies[min_idx]
eq_distance = dists[min_idx]

print(f"CH₄ ground state energy (VQE): {min_energy:.6f} Hartree")
print(f"Equilibrium C-H bond length: {eq_distance:.3f} Å")
print(f"Energy difference VQE-FCI: {(min_energy-fci_energies[min_idx])*1000:.2f} mHartree")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

from pyscf import gto, scf, fci

# Function to define molecular geometry for CH4
def get_ch4_coords(bond_length):
    """
    Returns coordinates for CH₄ (methane) molecule with tetrahedral geometry.
    The bond length is varied for all C-H bonds.
    """
    coords = [
        ["C", [0.0, 0.0, 0.0]],  # Carbon at origin
        ["H", [bond_length, 0.0, 0.0]],  # Hydrogen 1
        ["H", [-bond_length / 2, bond_length * (3 ** 0.5) / 2, 0.0]],  # Hydrogen 2
        ["H", [-bond_length / 2, -bond_length * (3 ** 0.5) / 2, 0.0]],  # Hydrogen 3
        ["H", [0.0, 0.0, bond_length]],  # Hydrogen 4
    ]
    return coords

# Function to generate the PySCF molecule
def get_pyscf_mol(coords, charge, spin, basis):
    """
    Create a PySCF molecule object with the given geometry, charge, spin, and basis set.
    """
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")
    mol = gto.Mole()
    mol.atom = atom_spec
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    return mol

# Function to generate the Qiskit problem
def get_qiskit_problem(coords, charge, spin, basis):
    """
    Generate the Qiskit Nature problem for the given geometry, charge, spin, and basis set.
    """
    atom_spec = ""
    for atom in coords:
        atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
    atom_spec = atom_spec.strip("; ")
    driver = PySCFDriver(atom=atom_spec, unit=DistanceUnit.ANGSTROM, charge=charge, spin=spin, basis=basis)
    return driver.run()

# Function to run PySCF UHF calculation
def run_uhf_pyscf(mol):
    """
    Perform Unrestricted Hartree-Fock (UHF) calculation using PySCF.
    """
    mf = scf.UHF(mol)
    ehf = mf.kernel()
    return ehf

# Function to run PySCF FCI calculation
def run_fci_pyscf(mol):
    """
    Perform Full Configuration Interaction (FCI) calculation using PySCF.
    """
    mf = scf.RHF(mol).run()
    cisolver = fci.FCI(mol, mf.mo_coeff)
    efci, _ = cisolver.kernel()
    return efci

# Function to run VQE using Qiskit
def run_vqe_cpu(problem, mapper):
    """
    Perform VQE calculation using Qiskit with the UCCSD ansatz.
    """
    # Use standard Qiskit Estimator (CPU-based)
    estimator = Estimator()
    
    optimizer = COBYLA(maxiter=200)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)
    
    # Better initial point (small random values)
    np.random.seed(42)
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    energy = problem.interpret(result).total_energies[0].real
    return energy

# --- Main Calculation Setup ---
basis = '6-31G(d,p)'
charge = 0  # Neutral molecule
spin = 0    # Singlet state (closed-shell CH₄)
dists = np.linspace(0.4, 4.0, 50)  # C-H bond distances in Å

vqe_energies, uhf_energies, fci_energies = [], [], []

# Loop through different C-H bond lengths
for i, d in enumerate(dists):
    print(f"Calculating CH₄ at C-H bond length = {d:.2f} Å ({i+1}/{len(dists)})")
    coords = get_ch4_coords(d)
    
    # Classical reference calculations
    mol = get_pyscf_mol(coords, charge, spin, basis)
    uhf_energy = run_uhf_pyscf(mol)
    fci_energy = run_fci_pyscf(mol)
    
    # VQE calculation
    problem = get_qiskit_problem(coords, charge, spin, basis)
    mapper = ParityMapper()
    vqe_energy = run_vqe_cpu(problem, mapper)
    
    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)
    print(f"  VQE={vqe_energy:.6f} | UHF={uhf_energy:.6f} | FCI={fci_energy:.6f}")

# Plot potential energy curves
plt.figure(figsize=(9, 5))

# Plot curves
plt.plot(dists, vqe_energies, '-o', label='VQE', color='green', linewidth=1, markersize=4)
plt.plot(dists, uhf_energies, '-s', label='UHF', color='red', linewidth=1, markersize=4)
plt.plot(dists, fci_energies, '--', label='FCI', color='black', linewidth=1)

plt.xlabel("C-H Bond Length (Å)", fontsize=12)
plt.ylabel("Energy (Hartree)", fontsize=12)
plt.title("CH₄ Potential Energy Surface (STO-3G)", fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Find minimum energy and corresponding bond length
min_idx = np.argmin(vqe_energies)
min_energy = vqe_energies[min_idx]
eq_distance = dists[min_idx]

print(f"CH₄ ground state energy (VQE): {min_energy:.6f} Hartree")
print(f"Equilibrium C-H bond length: {eq_distance:.3f} Å")
print(f"Energy difference VQE-FCI: {(min_energy-fci_energies[min_idx])*1000:.2f} mHartree")